# Components


In [ ]:
#load environment variables from .env file
import os 
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")




## langchain

In [ ]:
#Configure the Groq LLM
from langchain_groq import ChatGroq 

llm = ChatGroq(model_name="llama-3.1-8b-instant",temperature=0.7)
# uncomment to validate the LLM configuration
# llm.invoke("What is the capital of Nepal?") 

In [ ]:
#Example of messages format in LangGraph
from langchain_core.messages import AIMessage,HumanMessage

messages = [AIMessage(content=f'I am LangGraph Expert. I can help you with your queries related to LangGraph.',name='LLM')]
messages.append(HumanMessage(content=f'What is LangGraph?',name='User'))

#print the messages
for message in messages:
    message.pretty_print()


## Router and Tool
### Router helps selecting next step based on the user query
### Tool helps performing specific task

In [ ]:
def substract(a:int,b:int)->int:
    """
   Subsctract b from a and return the result.
   Args:
       a (int): The first number.
       b (int): The second number.
   Returns:
       int: The result of a - b.
    """
    return a-b

In [ ]:
### Binding tools 
tools=llm.bind_tools([substract])
messages.append(AIMessage(content=f"You are a helpful assistant.Do not assume the question, if you don't know the answer, just say it so",name='LLM'))

tool_call=tools.invoke("Can you subtract 10 from 20?")
print(tool_call.content) 
print(tool_call.tool_calls)


llm_call=tools.invoke("How are you?")
print(llm_call.content)
print(llm_call.tool_calls)

## Creating state
 - You may try to use data class or pydantic for state

In [ ]:
from typing import TypedDict
from typing import Annotated
from langgraph.graph.message import AnyMessage,add_messages

# annotated will show all the message in the list
class State(TypedDict):
    messages: Annotated[list[AnyMessage],add_messages]

In [ ]:
ai_message=AIMessage(content=f"What is the answer",name='LLM')

# function of reducer : append the ai_message to the messages
add_messages(messages,ai_message)

In [ ]:
def llm_call(state:State):
    return {"messages":[tools.invoke(state["messages"])]}

In [ ]:
# Build State Graph
from IPython.display import display,Image
from langgraph.graph import StateGraph,START,END

build_graph=StateGraph(State)

build_graph.add_node("llm_call",llm_call)  
build_graph.add_edge(START,"llm_call")
build_graph.add_edge("llm_call",END)

graph=build_graph.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# invoke the graph
messages=graph.invoke({"messages":"What is 10 minus 2"})

for message in messages['messages']:
    message.pretty_print()

In [ ]:
from langgraph.prebuilt import ToolNode,tools_condition

build_graph=StateGraph(State)
tool=[substract]

#add nodes
build_graph.add_node("llm_call",llm_call)  
build_graph.add_node("tools",ToolNode(tool))

#add edges
build_graph.add_edge(START,"llm_call")
build_graph.add_conditional_edges("llm_call",tools_condition)
build_graph.add_edge("tools",END)

graph=build_graph.compile()
display(Image(graph.get_graph().draw_mermaid_png()))


In [ ]:
# invoke the graph
messages=graph.invoke({"messages":"What is 10 minus 2"})

for message in messages['messages']:
    message.pretty_print()

In [ ]:
# invoke the graph
messages=graph.invoke({"messages":"What is the capital of the world?"})

for message in messages['messages']:
    message.pretty_print()

## Multiple Tools Integration

In [ ]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper 


In [ ]:
arxiv_wrapper_api=ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper_api)
print(arxiv.name)
arxiv.invoke("Love thy neighbour")

In [ ]:
wiki_wrapper_api=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=500)
wiki=WikipediaQueryRun(api_wrapper=wiki_wrapper_api)
print(wiki.name)
wiki.invoke("Who is Jesus Christ.")

In [ ]:
#tools list
tools=[arxiv,wiki]

#bind it with llm
llm_with_tools=llm.bind_tools(tools)

In [ ]:
llm_with_tools.invoke([HumanMessage(content=f"Who is Jesus Christ?") ])

In [ ]:
llm_with_tools.invoke([HumanMessage(content=f"Any research on 'Love thy neighbour'") ])

In [ ]:
llm_with_tools.invoke([HumanMessage(content=f"Who are you?") ])

In [ ]:
def tool_calling_llm(state:State):
    return {"messages":[llm_with_tools.invoke(state["messages"])]}
    

In [ ]:
#Node
graph = StateGraph(State)

#add nodes
graph.add_node("tool_calling_llm",tool_calling_llm)  
graph.add_node("tools",ToolNode(tools))

#add edges
graph.add_edge(START,"tool_calling_llm")
graph.add_conditional_edges("tool_calling_llm",tools_condition)
graph.add_edge("tools",END)

graph=graph.compile()
display(Image(graph.get_graph().draw_mermaid_png()))


In [ ]:
messages=graph.invoke({"messages":HumanMessage(content=f"Who is Jesus Christ?")})
for message in messages['messages']:
    message.pretty_print()

In [ ]:
messages=graph.invoke({"messages":HumanMessage(content=f"Who are you?")})
for message in messages['messages']:
    message.pretty_print()

In [ ]:
messages=graph.invoke({"messages":HumanMessage(content=f"Any research on existinence of Aliens?")})
for message in messages['messages']:
    message.pretty_print()

### ReAct Agent Architecture
- Used to develop complex Agent
- ReAct : General Agent architecure
    - Act: The model based on specific input calls specific tool
    - Observe: Passes tool output back to the model
    - Reason: The model will reason based on the output response from the tool to make next step


In [ ]:
# Custom functions
def multiply(a:int, b:int)->int:
    """Multiplys two numbers

    Args:
        a (int): first number
        b (int): second number

    Returns:
        int: result
    """
    return a*b

def add(a: int, b: int) -> int:
    """Adds a and b"""
    return a + b

In [ ]:
#tools list
tools=[arxiv,wiki,multiply,add]

#bind it with llm
llm_with_tools=llm.bind_tools(tools)
# llm_with_tools.invoke([HumanMessage(content=f"Any research on existinence of Aliens?") ])

# llm_with_tools.invoke([HumanMessage(content=f"Who is Nelson Mandela?") ])
# llm_with_tools.invoke([HumanMessage(content=f"Multilply 2 by 3") ])
# llm_with_tools.invoke([HumanMessage(content=f"Add 2 and 4")])
llm_with_tools.invoke([HumanMessage(content=f"Who are you?")])

In [ ]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage],add_messages]
    
def tool_calling_llm(state:State):
    return {"messages":[llm_with_tools.invoke(state["messages"])]}

# Build graph
graph = StateGraph(State)

# Add nodes
graph.add_node("tool_calling_llm",tool_calling_llm)
graph.add_node("tools",ToolNode(tools))

# Add edges        
graph.add_edge(START,"tool_calling_llm")
graph.add_conditional_edges("tool_calling_llm",tools_condition)
graph.add_edge("tools","tool_calling_llm")

graph=graph.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
messages=graph.invoke({"messages":HumanMessage(content=f"Any research on existinence of Aliens then summarize in 1 sentence?")})
for message in messages['messages']:
    message.pretty_print()

messages=graph.invoke({"messages":HumanMessage(content=f"Who are you?")})
for message in messages['messages']:
    message.pretty_print()

### Memory

- Can automatically save graph state after each step by using checkpointer
- Built in persistence layer allows langgraph to pick up from the last state update
- Easier checkpoint to use is MemorySaver, an in memory key-value store for Graph State
- All we need to do is simple compile graph with a checkointer

Documentation ref: https://langchain-ai.github.io/langgraph/concepts/persistence/#get-state-history

In [ ]:
messages=graph.invoke({"messages":HumanMessage(content=f"Add 5 and 6.")})
for message in messages['messages']:
    message.pretty_print()

In [ ]:
messages=graph.invoke({"messages":HumanMessage(content=f"Then add 6 to the result.If you lack context, just say so.")})
for message in messages['messages']:
    message.pretty_print()

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# Build graph with memory
graph_memory = StateGraph(State)

# Add nodes
graph_memory.add_node("tool_calling_llm",tool_calling_llm)
graph_memory.add_node("tools",ToolNode(tools))

# Add edges        
graph_memory.add_edge(START,"tool_calling_llm")
graph_memory.add_conditional_edges("tool_calling_llm",tools_condition)
graph_memory.add_edge("tools","tool_calling_llm")

#Initialize Memory
memory=MemorySaver()

graph_memory=graph_memory.compile(checkpointer=memory)
display(Image(graph_memory.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "1"}}

messages=graph_memory.invoke({"messages":HumanMessage(content=f"Add 5 and 6.")},config=config)
for message in messages['messages']:
    message.pretty_print()
    
    

In [ ]:
messages=graph_memory.invoke({"messages":HumanMessage(content=f"Add 5 and 6.")},config=config)
for message in messages['messages']:
    message.pretty_print()
    
    

In [ ]:
messages=graph_memory.invoke({"messages":HumanMessage(content=f"Add 6 to the output. If you can't do it, just say it so.")},config=config)
for message in messages['messages']:
    message.pretty_print()

## Streaming
 - stream()
    
      Synchronous streaming

 - astream()
    
      Asynchronous streaming

In [ ]:
config = {"configurable": {"thread_id": "2"}}
for chunk in graph_memory.stream({"messages":HumanMessage(content=f"Add 2 to the output. If you can't do it, just say it so.")},config=config,stream_mode="updates"):
    print(chunk)

In [ ]:
for chunk in graph_memory.stream({"messages":HumanMessage(content=f"Add 2 to the output. If you can't do it, just say it so.")},config=config,stream_mode="values"):
    print(chunk)

In [ ]:
config = {"configurable": {"thread_id": "3"}}
inputs = {"messages": [HumanMessage(content="Add 2 to the output. If you can't do it, just say it so.")]}
async for event in graph_memory.astream_events(inputs, config, version="v2"):
    print(event)
